In [ ]:
# =========================
# Imports
# =========================
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    roc_auc_score, average_precision_score, log_loss,
    precision_score, recall_score, f1_score
)

import mlflow
import mlflow.lightgbm
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from lightgbm import LGBMClassifier, LGBMRanker, early_stopping, log_evaluation

# Classifiers supplémentaires
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

MLFLOW_TRACKING_URI = "http://localhost:5555"
EXP_NAME = "stockout_substitution_hyperopt_classifier_ranker"

# =========================
# Fonction utilitaire : racine projet
# =========================
def find_project_root(marker=".git"):
    path = Path().resolve()
    while path != path.parent:
        if (path / marker).exists():
            return path
        path = path.parent
    raise FileNotFoundError(f"Project root with {marker} not found")

ROOT_DIR = find_project_root()
DATA_RAW_DIR = ROOT_DIR / "data" / "raw"

# =========================
# Chargement des données
# =========================
substitutions = pd.read_csv(DATA_RAW_DIR / "substitutions" / "raw_substitutions_substitutions.csv")
produits = pd.read_csv(DATA_RAW_DIR / "produits" / "raw_produits_produits.csv")
transactions = pd.read_csv(DATA_RAW_DIR / "transactions" / "raw_transactions_transactions.csv")

# =========================
# Construction dataset + features
# =========================
def merge_and_add_suffix(df_add_suffix, df_keep, suffix, column_to_merge):
    df_add = df_add_suffix.copy()
    df_add = df_add.rename(columns={c: c + suffix for c in df_add.columns if c != column_to_merge})
    return pd.merge(df_keep, df_add, left_on=column_to_merge, right_on=column_to_merge, how='left')

def build_dataset(transactions, substitutions, produits):
    subs_prod_orig = merge_and_add_suffix(produits, substitutions, 'Original', 'idProduitOriginal')
    subs_prod_orig_subst = merge_and_add_suffix(produits, subs_prod_orig, 'Substitution', 'idProduitSubstitution')
    df = pd.merge(transactions, subs_prod_orig_subst,
                  left_on=['idProduit','idTransaction'],
                  right_on=['idProduitOriginal','idTransaction'], how='inner')
    df['estAcceptee_bin'] = (~df['estAcceptee']).astype(int)
    df['date'] = pd.to_datetime(df['dateHeureTransaction'])
    df['Month'] = df['date'].dt.month
    df['Day_of_week_name'] = df['date'].dt.day_name()
    df["MemeMarque"] = (df["marqueOriginal"] == df["marqueSubstitution"]).astype(int)
    df["MemeNutriscore"] = (df["nutriscoreOriginal"] == df["nutriscoreSubstitution"]).astype(int)
    df["MemeConditionnement"] = (df["conditionnementOriginal"] == df["conditionnementSubstitution"]).astype(int)
    df["MemeTypeMarque"] = (df["typeMarqueOriginal"] == df["typeMarqueSubstitution"]).astype(int)
    df["DiffPrix"] = df["prixSubstitution"] - df["prixOriginal"]
    df["MemeBio"] = ((df["estBioOriginal"] == True) & (df["estBioSubstitution"] == True)).astype(int)
    return df

df = build_dataset(transactions, substitutions, produits)

# =========================
# Préprocessing
# =========================
features_num = ["DiffPrix", "MemeMarque", "MemeNutriscore", "MemeBio",
                "prixOriginal", "MemeConditionnement", "MemeTypeMarque", "estBioOriginal", "Month"]
features_cat = ["categorieOriginal", "marqueOriginal", "typeMarqueOriginal", "nutriscoreOriginal",
                "origineOriginal", "conditionnementOriginal", "categorieSubstitution",
                "typeMarqueSubstitution", "origineSubstitution", "Day_of_week_name"]

X = df[features_num + features_cat]
y = df["estAcceptee_bin"]

# Split temporel
cutoff_idx = int(len(df) * 0.8)
X_train_raw, X_val_raw = X.iloc[:cutoff_idx], X.iloc[cutoff_idx:]
y_train, y_val = y.iloc[:cutoff_idx], y.iloc[cutoff_idx:]

# Pipeline
numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer([
    ("num", numeric_transformer, features_num),
    ("cat", categorical_transformer, features_cat)
])

X_train = preprocessor.fit_transform(X_train_raw)
X_val = preprocessor.transform(X_val_raw)

# =========================
# Calcul des groups pour ranking
# =========================
group_train = df.iloc[:cutoff_idx].groupby('idTransaction').size().to_numpy()
group_val = df.iloc[cutoff_idx:].groupby('idTransaction').size().to_numpy()

# =========================
# MLflow setup
# =========================
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)  # ou localhost si accessible

try:
    mlflow.create_experiment(EXP_NAME)
except mlflow.exceptions.MlflowException:
    pass
mlflow.set_experiment(EXP_NAME)

# =========================
# Métriques
# =========================
def compute_classification_metrics(y_true, y_proba, threshold=0.5):
    y_pred = (y_proba >= threshold).astype(int)
    return {
        "auc": float(roc_auc_score(y_true, y_proba)) if len(np.unique(y_true)) > 1 else np.nan,
        "pr_auc": float(average_precision_score(y_true, y_proba)),
        "logloss": float(log_loss(y_true, y_proba)),
        "precision": float(precision_score(y_true, y_pred, zero_division=0)),
        "recall": float(recall_score(y_true, y_pred, zero_division=0)),
        "f1": float(f1_score(y_true, y_pred, zero_division=0)),
    }

def compute_ranking_metrics(y_true, y_score, group, ks=(1,3,5)):
    metrics = {}
    idx = 0
    per_k = {k: {"ndcg": [], "hit": []} for k in ks}
    for g in group:
        y_g = np.asarray(y_true[idx: idx + g])
        s_g = np.asarray(y_score[idx: idx + g])
        idx += g
        order = np.argsort(-s_g)
        rels_sorted = y_g[order]
        for k in ks:
            rels_k = rels_sorted[:k]
            per_k[k]["ndcg"].append(np.sum(rels_k / np.log2(np.arange(2, len(rels_k)+2))))
            per_k[k]["hit"].append(float(np.any(rels_k > 0)))
    for k in ks:
        metrics[f"ndcg_at_{k}"] = np.mean(per_k[k]["ndcg"])
        metrics[f"hit_rate_at_{k}"] = np.mean(per_k[k]["hit"])
    return metrics

# =========================
# Dictionnaire des modèles
# =========================
models = {
    "LogReg": {"model_class": LogisticRegression, "param_grid": {"C": [0.1,1.0,10.0], "penalty":["l2"]}, "fixed_params": {"solver":"lbfgs","max_iter":2000,"n_jobs":-1,"random_state":42}, "type":"classification"},
    "XGBClassifier": {"model_class": XGBClassifier, "param_grid": {"n_estimators":[500,1000],"max_depth":[4,6,8],"learning_rate":[0.03,0.05,0.1],"subsample":[0.7,0.9,1.0],"colsample_bytree":[0.7,0.9,1.0],"min_child_weight":[1,5,10],"reg_alpha":[0.0,0.1,1.0],"reg_lambda":[1.0,2.0,5.0]}, "fixed_params":{"objective":"binary:logistic","eval_metric":"auc","tree_method":"hist","random_state":42,"n_jobs":-1}, "type":"classification"},
    "LGBMClassifier": {"model_class": LGBMClassifier, "param_grid":{"num_leaves":[31,63,127],"learning_rate":[0.03,0.05,0.1],"n_estimators":[500,1000],"min_child_samples":[20,50,100],"subsample":[0.7,0.9,1.0],"colsample_bytree":[0.7,0.9,1.0],"reg_alpha":[0.0,0.1,1.0],"reg_lambda":[0.0,0.1,1.0]}, "fixed_params":{"objective":"binary","metric":"auc","random_state":42,"n_jobs":-1}, "type":"classification"},
    "CatBoostClassifier": {"model_class": CatBoostClassifier, "param_grid":{"depth":[6,8,10],"learning_rate":[0.03,0.05,0.1],"iterations":[500,1000],"l2_leaf_reg":[1,3,5,9],"subsample":[0.7,0.9,1.0],"rsm":[0.7,0.9,1.0]}, "fixed_params":{"loss_function":"Logloss","eval_metric":"AUC","random_seed":42,"verbose":0}, "type":"classification"},
    "LGBMRanker": {"model_class": LGBMRanker, "param_grid":{"num_leaves":[31,63,127],"learning_rate":[0.03,0.05,0.1],"n_estimators":[500,1000],"min_child_samples":[20,50,100],"subsample":[0.7,0.9,1.0],"colsample_bytree":[0.7,0.9,1.0]}, "fixed_params":{"objective":"lambdarank","metric":"ndcg","random_state":42,"n_jobs":-1}, "type":"ranking"}
}

# =========================
# Wrappers Hyperopt
# =========================
def objective_classifier(params, model_class, X_train, y_train, X_val, y_val):
    params = {k:int(v) if isinstance(v,float) and v.is_integer() else v for k,v in params.items()}
    with mlflow.start_run(nested=True):
        model = model_class(**params)
        model.fit(X_train, y_train)
        y_proba = model.predict_proba(X_val)[:,1]
        metrics = compute_classification_metrics(y_val, y_proba)
        mlflow.log_params(params)
        mlflow.log_metrics(metrics)
        if isinstance(model,LGBMClassifier):
            mlflow.lightgbm.log_model(model,"model")
        return {"loss": -metrics["auc"], "status": STATUS_OK}

def objective_ranker(params, X_train, y_train, X_val, y_val, group_train, group_val):
    params = {k:int(v) if k in ["num_leaves","n_estimators","min_child_samples"] else v for k,v in params.items()}
    with mlflow.start_run(nested=True):
        model = LGBMRanker(**params)
        model.fit(
            X_train, y_train,
            group=group_train,
            eval_set=[(X_val, y_val)],
            eval_group=[group_val],
            eval_metric="ndcg",
            callbacks=[early_stopping(stopping_rounds=50, verbose=False),
                       log_evaluation(period=0)]
        )
        best_iter = model.best_iteration_
        mlflow.log_metric("best_iteration", best_iter)
        scores = model.predict(X_val, num_iteration=best_iter)
        metrics = compute_ranking_metrics(y_val, scores, group_val)
        mlflow.log_params(params)
        mlflow.log_metrics(metrics)
        mlflow.lightgbm.log_model(model, "model")
        return {"loss": -metrics["ndcg_at_3"], "status": STATUS_OK}

# =========================
# Boucle sur tous les modèles
# =========================
for name, cfg in models.items():
    print(f"=== Optimisation {name} ===")
    # Création de l'espace Hyperopt
    space = {k: hp.choice(k, v) if isinstance(v,list) else v for k,v in cfg["param_grid"].items()}
    trials = Trials()
    if cfg["type"]=="classification":
        fmin(fn=lambda p: objective_classifier(p, cfg["model_class"], X_train, y_train, X_val, y_val),
             space=space, algo=tpe.suggest, max_evals=20, trials=trials, rstate=np.random.default_rng(42))
    else:
        fmin(fn=lambda p: objective_ranker(p, X_train, y_train, X_val, y_val, group_train, group_val),
             space=space, algo=tpe.suggest, max_evals=20, trials=trials, rstate=np.random.default_rng(42))


=== Optimisation LogReg ===
  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

/home/eric/.cache/pypoetry/virtualenvs/algo-reco-Zao1WwXe-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(



🏃 View run likeable-crab-176 at: http://localhost:5555/#/experiments/3/runs/e9e362a70ea34426ab803bd1de1f040a

🧪 View experiment at: http://localhost:5555/#/experiments/3

  5%|▌         | 1/20 [00:00<00:08,  2.31trial/s, best loss: -0.6990719003179975]

/home/eric/.cache/pypoetry/virtualenvs/algo-reco-Zao1WwXe-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(



🏃 View run whimsical-hen-705 at: http://localhost:5555/#/experiments/3/runs/e312a4cea3e541299c6562261b682d8b

🧪 View experiment at: http://localhost:5555/#/experiments/3                     

 10%|█         | 2/20 [00:00<00:07,  2.36trial/s, best loss: -0.6990719003179975]

...

🏃 View run blushing-mouse-673 at: http://localhost:5555/#/experiments/3/runs/c48c74c047db43e2bc80fd40db989b9a

🧪 View experiment at: http://localhost:5555/#/experiments/3                     

[LightGBM] [Info] Number of positive: 50659, number of negative: 40282           
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002971 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 826                                                 
[LightGBM] [Info] Number of data points in the train set: 90941, number of used features: 154
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.557053 -> initscore=0.229212   
[LightGBM] [Info] Start training from score 0.229212                             
 90%|█████████ | 18/20 [01:12<00:07,  3.99s/trial, best loss: -0.724539654385693]

/home/eric/.cache/pypoetry/virtualenvs/algo-reco-Zao1WwXe-py3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(

2026/01/06 23:02:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



🏃 View run upbeat-wren-425 at: http://localhost:5555/#/experiments/3/runs/e5f194c414af4cbd8a7688cb3a870802

🧪 View experiment at: http://localhost:5555/#/experiments/3                     

[LightGBM] [Info] Number of positive: 50659, number of negative: 40282           
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002483 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 826                                                 
[LightGBM] [Info] Number of data points in the train set: 90941, number of used features: 154
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.557053 -> initscore=0.229212   
[LightGBM] [Info] Start training from score 0.229212                             
 95%|█████████▌| 19/20 [01:15<00:03,  3.85s/trial, best loss: -0.724539654385693]

/home/eric/.cache/pypoetry/virtualenvs/algo-reco-Zao1WwXe-py3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(

2026/01/06 23:02:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



🏃 View run able-crane-464 at: http://localhost:5555/#/experiments/3/runs/14fd2f564cac4432a49ba71e45b4e6c6

🧪 View experiment at: http://localhost:5555/#/experiments/3                     

100%|██████████| 20/20 [01:21<00:00,  4.07s/trial, best loss: -0.724539654385693]
=== Optimisation CatBoostClassifier ===
0:	learn: 0.6822086	total: 75.3ms	remaining: 37.6s    

1:	learn: 0.6726672	total: 94.4ms	remaining: 23.5s    

2:	learn: 0.6644129	total: 114ms	remaining: 18.9s     

3:	learn: 0.6567553	total: 133ms	remaining: 16.5s     

4:	learn: 0.6504622	total: 156ms	remaining: 15.4s     

5:	learn: 0.6444113	total: 176ms	remaining: 14.5s     

6:	learn: 0.6387959	total: 195ms	remaining: 13.8s     

7:	learn: 0.6340856	total: 214ms	remaining: 13.2s     

8:	learn: 0.6300859	total: 236ms	remaining: 12.9s     

9:	learn: 0.6262142	total: 257ms	remaining: 12.6s     

10:	learn: 0.6230159	total: 276ms	remaining: 12.3s    

11:	learn: 0.6201667	total: 317ms	remaining: 12.9s    

12:	learn: 0.6175